#Contents
* [Load up dependencies](#Load-dependencies)
* [Parsing & Structuring Data](#Parsing-&-Structuring-Data)
* [Load the matrices and vectors from temp file](#Load-the-matrices-and-vectors)
* [Features](#Featurization)
* [The Model](#Building-the-model)
* [Results](#Running-Model-on-Test-Set)

#Load dependencies

In [1]:
import json
from pprint import pprint
import nltk, re
from nltk import word_tokenize
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

#Parsing & Structuring Data

In [2]:
# import user data from json file
udata = []
with open('yelp_academic_dataset_user.json') as user_data:    
    for line in user_data:
        temp_user = json.loads(line)
        # remove users with less than 20 reviews
        # users with < 20 reviews are almost certainly not elite
        if temp_user['review_count'] >= 20:
            # only keep useful attributes
            temp_user = {k:temp_user[k] for k in temp_user if k in ['elite', 'review_count', 'user_id']}
            udata.append(temp_user)

In [3]:
# check user data (non-essential code)
udata[0]

# note: the 'elite' attribute is a list of years

{'elite': [2005, 2006],
 'review_count': 108,
 'user_id': '18kPq7GPye-YQ3LyKyAZPw'}

In [4]:
# import review data from json file
rdata = []
with open('yelp_academic_dataset_review.json') as review_data:
    for line in review_data:
        temp_review = json.loads(line)
        # only keep useful attributes
        temp_review = {k:temp_review[k] for k in temp_review if k not in ['review_id', 'stars', 'type']}
        rdata.append(temp_review)

In [5]:
# check review data (non-essential code)
rdata[0]

{'business_id': 'vcNAWiLM4dR7D2nwwJ7nCA',
 'date': '2007-05-17',
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
 'user_id': 'Xqd0DzHaiyRqVH3WRG7hzg',
 'votes': {'cool': 1, 'funny': 0, 'useful': 2}}

In [6]:
# import business ids from json file
b_ids = []
with open('yelp_academic_dataset_business.json') as business_data:
    for line in business_data:
        temp_biz = json.loads(line)
        if temp_biz['state'] in ['AZ', 'NV', 'WI', 'IL', 'NC', 'PA']:    # only add businesses in the US (exclude Canada & Germany)
            b_ids.append(temp_biz['business_id'])                        # we only need the business ids

In [7]:
# check business ids (non-essential code)
b_ids[0]

'vcNAWiLM4dR7D2nwwJ7nCA'

####Structuring reviews by year, then by user

In [8]:
# build a dictionary for predicting elite years with
# key: user id
# value: [years in which their reviews let to elite status the following year]
elite_review_years_dict = {}
for user in udata:
    elite_review_years_dict[user['user_id']] = [year-1 for year in user['elite']]

In [10]:
# build a reviews dictionary with 
# key: (user_id, year) and 
# value: [review objects]
reviews_dict = {}
for review in rdata:
    user_id = review['user_id']
    if review['business_id'] in b_ids and user_id in elite_review_years_dict:
        # exclude 2015 reviews since we don't know if they'll lead to elite in 2016 or not
        year = int(review['date'][:4])
        if year != 2015:
            if (user_id, year) in reviews_dict:
                reviews_dict[(user_id, year)].append(review)
            else:
                reviews_dict[(user_id, year)] = [review]

####Sorting out the test set reviews

In [11]:
# build sets of arrays such that
# one array contains the list of a given user's review objects during a given year
# and the other array contains an elite flag/label for that year of reviews
# (elite = 1, non-elite = 0)

# set aside 2014 reviews as the test set (roughly 20%) to see if we can predict on this most recent year after training

train_dev_set = []
train_dev_labels = []

test_set = []
test_labels = []

for id_year_tuple, reviews in reviews_dict.items():
    year = id_year_tuple[1]

    if year == 2014:     
        test_set.append(reviews)
        test_labels.append(1) if year in elite_review_years_dict[user_id] else test_labels.append(0)                
    else:       
        train_dev_set.append(reviews)
        train_dev_labels.append(1) if year in elite_review_years_dict[user_id] else train_dev_labels.append(0)

In [12]:
# sanity check: make sure the number of reviews and number of labels match for the train_dev set
assert len(train_dev_set) == len(train_dev_labels)
train_dev_set_len = len(train_dev_labels)
print(train_dev_set_len)

119469


In [13]:
# sanity check: make sure the number of reviews and number of labels match for the test set
assert len(test_set) == len(test_labels)
test_set_len = len(test_labels)
print(test_set_len)

34770


In [14]:
total_len = train_dev_set_len + test_set_len
print("Proportion of train/dev set to test set: "+str(train_dev_set_len/total_len))

Proportion of train/dev set to test set: 0.7745706338863712


####Randomize and divide training and development data

In [15]:
# convert to np array to randomize/shuffle data
np_train_dev_set = np.array(train_dev_set)
np_train_dev_labels = np.array(train_dev_labels)

In [16]:
rand_indices = np.random.permutation(len(train_dev_labels))
shuffled_train_dev_set = np_train_dev_set[rand_indices]
shuffled_train_dev_labels = np_train_dev_labels[rand_indices]

In [17]:
subset_length = len(shuffled_train_dev_set)
# 80-20 between training, and dev (within training+dev)
train_set = shuffled_train_dev_set[:subset_length*4//5]
train_set_labels = shuffled_train_dev_labels[:subset_length*4//5]

dev_set = shuffled_train_dev_set[subset_length*4//5:]
dev_set_labels = shuffled_train_dev_labels[subset_length*4//5:]

In [ ]:
# Check that the percentage of elite users is reasonable
def get_elite_ratio(labels):
    elite = 0
    for item in labels:
        if item == 1:
            elite += 1
    return elite / len(labels)
print("training set percentage elite: " +str(get_elite_ratio(train_set_labels)))
print("development set percentage elite: " +str(get_elite_ratio(dev_set_labels)))
print("development set percentage elite: " +str(get_elite_ratio(test_labels)))

####Create tokenized version of dataset

In [20]:
# convert dataset from: a list of lists of review objects (grouped by user-years)
# to: a list of (list of reviews => list of sentences => list of words)
def tokenize_dataset(dataset):
    tokenized_dataset = []
    for review_objs in dataset:
        tokenized_reviews = [nltk.word_tokenize(sent) for sent in sent_tokenizer.tokenize(review['text']) for review in review_objs]
        tokenized_dataset.append(tokenized_reviews)
    return tokenized_dataset

In [ ]:
tokenized_train_set = tokenize_dataset(train_set)
tokenized_dev_set   = tokenize_dataset(dev_set)

###Save this matrix and vector to a temp file, so info is loadable quickly

In [93]:
data_file = open('data_sets.npz2', 'wb+')
np.savez(data_file, train_set, train_set_labels, dev_set, dev_set_labels, tokenized_train_set, tokenized_dev_set, test_set, test_labels)
#np.savez(data_file, train_set, train_set_labels, dev_set, dev_set_labels, test_set, test_labels)
data_file.close()

#Load the matrices and vectors

In [94]:
# Remember to import dependencies first
load_data = np.load('data_sets.npz')

In [95]:
# arrays in order are
# train_set, train_set_labels, dev_set, dev_set_labels, tokenized_train_set, tokenized_dev_set, test_set, test_labels
load_data.files

['arr_4', 'arr_2', 'arr_5', 'arr_0', 'arr_1', 'arr_3']

In [96]:
train_set = load_data['arr_0']
train_set_labels = load_data['arr_1']
dev_set = load_data['arr_2']
dev_set_labels = load_data['arr_3']
tokenized_dataset = load_data['arr_4']
tokenized_dataset_labels = load_data['arr_5']
test_set = load_data['arr_6']
test_labels = load_data['arr_7']

In [98]:
assert len(train_set) == len(train_set_labels)
assert len(dev_set) == len(dev_set_labels)
assert len(tokenized_dataset) == len(tokenized_dataset_labels)
assert len(test_set) == len(test_labels)

# Featurization

####Feature development
Adding a new feature:
* Remember to step through each feature definition below
* Then define the feature function, if necessary
* Set your feature to some variable in the featurize() function
* Make sure feature is appended in featurize(): `feature_entry.append(NEW_FEATURE)`

In [5]:
def total_reviews(reviews_arr):
    return len(reviews_arr)

def basic_totals(reviews_arr):
    basic_arr = []
    
    chars = 0
    paragraphs = 0
    cool_votes = 0
    funny_votes = 0
    useful_votes = 0
    
    for review in reviews_arr:
        chars += len(review['text'])
        paragraphs += review['text'].count("\n\n")
        cool_votes += review['votes']['cool']
        funny_votes += review['votes']['funny']
        useful_votes += review['votes']['useful']
    
    basic_arr.append(chars)
    basic_arr.append(paragraphs)
    basic_arr.append(cool_votes)
    basic_arr.append(funny_votes)
    basic_arr.append(useful_votes)
    
    return basic_arr

In [6]:
def get_NLP_features(tokenized_reviews):
    NLP_features = []
    
    total_sents = 0
    total_words = 0
    vocabulary = set([])
    
    for review in tokenized_reviews:
        for sent in review:
            total_sents += 1
            total_words += len(sent) - 1
            for word in sent:
                if word.lower() not in vocabulary:
                    vocabulary.add(word.lower())

    vocabulary_size = len(vocabulary)
    
    return [total_sents, total_words, vocabulary_size]

In [ ]:
# these features were not as successful as we hoped

# average COUNT over all the user's reviews in the given year
    # personal_pronouns: i, me, my, mine, we, us, our
    # directed_pronouns: you, your, you're
    # other_pronouns: he, she, him, her, they, them
    # question_words: who, what, why, when, where, how
# percentage versions of the above

In [ ]:
# # for the purpose of the calculating the readability features
# # this code writes out each all of a user's reviews (within a year) to a single text file
# # so the readability script can write the features to a csv file
# # readability --csv --tokenizer='tokenizer -L en-u8 -P -S -E "" -N' users_train/*.txt >readability_measures_train*.csv

# count = 0    # to preserve indices with labels
# for user in train_set: 
#     data_file = open('users_train/' + '%06d' %(count) + '.txt', 'w')
#     for review in user: 
#         data_file.write(review['text'])
#         data_file.write('\n\n') # add a paragraph marker for free after each review
#     data_file.close()
#     count += 1
    
# count = 0    #to preserve order with labels
# for user in dev_set: 
#     data_file = open('users_dev/' + '%06d' %(count) + '.txt', 'w')
#     for review in user: 
#         data_file.write(review['text'])
#         data_file.write('\n\n') # add a paragraph marker for free after each review
#     data_file.close()
#     count += 1

In [ ]:
# for readability features only
# not in use for last iteration

# column indices for features
ARI = 1
COLMAN_LIAU = 2
FLESCHREADINGEASE = 3
GUNNINGFOGINDEX = 4
KINCAID = 5
LIX = 6
RIX = 7
SMOGINDEX = 8
ARTICLE = 9
AUXVERB = 10
CHARACTERS = 11
CHARACTERS_PER_WORD = 12
COMPELX_WORDS = 13
CONJUNCTION = 14
INTERROGATIVE = 15
LONG_WORDS = 16
NOMINALIZATION = 17
PARAGRAPHS = 18
PREPOSITION = 19
PRONOUN = 20
SENTENCES = 21
SENTENCES_PER_PARAGRAPH = 22
SUBORDINATION = 23
SYLL_PER_WORD = 24
SYLLABLES = 25
TOBEVERB = 26
TYPE_TOKEN_RATIO = 27
WORDS = 28
WORDS_PER_SENTENCE = 29
WORDTYPES = 30

import csv
featurized_matrix = []

def get_readability_features():
    # remove first column (filename)
    # remove first row (column names header) for every file
    for i in range(0,11): # this loop because argument list was too long; had to chop into pieces
        f = open('readability_measures_train_dev_' + '%06d' %((i+1)*10000) + '.csv', 'r')
        reader = csv.reader(f)
        for row in reader:
            if not (row[0] == ''):
                # choose which features to add by index here:
                featurized_matrix.append([float(x) for x in row[ARI:SMOGINDEX+1]]) 
    f = open('readability_measures_train_dev_119469.csv', 'r') # last file had a unique filename
    reader = csv.reader(f)
    for row in reader:
        if not row[0] == '':
            featurized_matrix.append([float(x) for x in row[ARI:SMOGINDEX+1]])
    f.close()

In [7]:
def featurize(dataset, tokenized_dataset):
    feature_array = []
    index = 0
    
    for reviews_arr in dataset:
        feature_entry = []

        review_count = total_reviews(reviews_arr)
        totals = basic_totals(reviews_arr)
        
        totals.extend(get_NLP_features(tokenized_dataset[index]))
        
        averages = [total/review_count for total in totals]

        feature_entry.append(review_count)
        feature_entry.extend(totals)
        feature_entry.extend(averages)
        
        feature_array.append(feature_entry)
        index += 1
    #feature_array.extend(get_readability())
    return feature_array

In [50]:
# so we can print out which features were most helpful later on
feature_dict = {
    1: "total reviews",
    2: "total characters",
    3: "total paragraphs",
    4: "total cool votes",
    5: "total funny votes",
    6: "total useful votes",
    7: "total sentences",
    8: "total words",
    9: "total size of vocabulary (unique words)",
    10: "chars per review",
    11: "paragraphs per review",
    12: "cool votes per review",
    13: "funny votes per review",
    14: "useful votes per review",
    15: "sentences per review",
    16: "words per review",
    17: "size of vocabulary per review"
}

####Apply Features

In [8]:
% time featurized_train = featurize(train_set, tokenized_train_set)

Wall time: 1min 17s


In [9]:
% time featurized_dev = featurize(dev_set, tokenized_train_set)

Wall time: 18.1 s


In [12]:
#tokenized_test_set = tokenize_dataset(test_set)

In [13]:
% time featurized_test = featurize(test_set, tokenized_test_set)

Wall time: 16.4 s


In [14]:
% time normalized_train = normalize(featurized_train, norm='l2')

Wall time: 322 ms


In [15]:
% time normalized_dev = normalize(featurized_dev, norm='l2')

Wall time: 71.6 ms


In [16]:
% time normalized_test = normalize(featurized_test, norm='l2')

Wall time: 119 ms


####Create subset for faster training

In [24]:
one_index = np.where(train_set_labels == 1)[0]
zero_index = np.where(train_set_labels == 0)[:len(one_index)][0]

subset_train = normalized_train[list(one_index)+list(zero_index)]
subset_labels = train_set_labels[list(one_index)+list(zero_index)]

In [12]:
# printing again for reference
print("training set percentage elite: " +str(get_elite_ratio(train_set_labels)))
print("development set percentage elite: " +str(get_elite_ratio(dev_set_labels)))

training set percentage elite: 0.23506147004969918
development set percentage elite: 0.23492236219813334


#Building the model

In [19]:
# print out our results with this
def get_model_stats(dev_set, dev_set_labels, clf):
    true_pos = 0
    false_pos = 0
    true_neg = 0
    false_neg = 0
    total = 0
    
    for element in dev_set:
        correct_label = dev_set_labels[total]
        
        if clf.predict(element) == correct_label:
            if correct_label == 0:
                true_neg += 1
            else:
                true_pos += 1
        else:
            if correct_label == 0:
                false_pos += 1
            else:
                false_neg += 1
        total += 1
        
    accuracy = (true_pos + true_neg) / total
    precision = true_pos / (true_pos + false_pos)
    recall = true_pos / (true_pos + false_neg)
    f1_score = 2 * ((precision * recall) / (precision + recall))
    
    print("T Positive: %s, F Positive: %s" % (true_pos, false_pos))
    print("F Negative: %s, T Negative: %s" % (false_neg, true_neg))
    print()
    print("The following metrics are on a scale of 0 to 1:")
    print("Model accuracy: "+str(accuracy))
    print("Model precision: "+str(precision))
    print("Model recall: "+str(recall))
    print("Model F1 Score: "+str(f1_score))

##Gaussian Naive Bayes Model

We can see that a naive bayes model has quite low accuracy as well as precision. Its accuracy is lower than always guessing non-elite, and its precision of 29.47% means that out of the times it guesses elite, it is only correct that percentage of the time, which is definitely not great.

###Normalized Features Attempt

In [113]:
gaussian_clf = GaussianNB()
% time gaussian_clf.fit(normalized_train, train_set_labels)

Wall time: 71.1 ms


GaussianNB()

In [116]:
% time get_model_stats(normalized_dev, dev_set_labels, gaussian_clf)

T Positive: 2296, F Positive: 5496
F Negative: 3317, T Negative: 12784

The following metrics are on a scale of 0 to 1:
Model accuracy: 0.6311471979240781
Model precision: 0.2946611909650924
Model recall: 0.40905041867094244
Model F1 Score: 0.34255874673629244
Wall time: 2.33 s


###Non-normalized Features Attempt

In [117]:
gaussian_clf = GaussianNB()
% time gaussian_clf.fit(featurized_train, train_set_labels)

Wall time: 393 ms


GaussianNB()

In [118]:
% time get_model_stats(featurized_dev, dev_set_labels, gaussian_clf)

T Positive: 1001, F Positive: 761
F Negative: 4612, T Negative: 17519

The following metrics are on a scale of 0 to 1:
Model accuracy: 0.7751224207927008
Model precision: 0.5681044267877412
Model recall: 0.17833600570105113
Model F1 Score: 0.2714576271186441
Wall time: 3.02 s


##SVM Model

###Non-normalized Features Attempt

An SVM-based model was prohibitivly slow to run and not particularly accurate, so we decided against using an SVM Model. The lack of wall time for the learning machine fitting was a result of us changing the featurization a little bit, which caused the machine to take over 45 minutes to run, at which point we just stopped the kernel and decided not to proceed with this model.

In [ ]:
svm_clf = svm.SVC()
% time svm_clf.fit(featurized_train, train_set_labels)

In [25]:
% time get_model_accuracy(featurized_dev, dev_set_labels, svm_clf)

Model accuracy: 76.34453605658561 percent.
Wall time: 57.9 s


##Logistic Regression Model

###Subset Normalized Features Attempt

In [25]:
logreg_clf = linear_model.LogisticRegression(C=1)
% time logreg_clf.fit(subset_train, subset_labels)

Wall time: 626 ms


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

We can see that a linear logistic regression model performs **very poorly**: it has an accuracy of 77.27 percent _when run on the data on which it was trained_. We know that the dev set has 0.2349 (23.49%) of its data labeled "non-elite", so by guessing "non-elite" every time, it will achieve 76.51% accuracy. As we see that it is a poor match when run on that data on which it was trained, we know then that this model must be lacking.

In [26]:
% time get_model_stats(subset_train, subset_labels, logreg_clf)

T Positive: 1206, F Positive: 456
F Negative: 21260, T Negative: 72653

The following metrics are on a scale of 0 to 1:
Model accuracy: 0.7727857703374313
Model precision: 0.7256317689530686
Model recall: 0.05368111813406926
Model F1 Score: 0.09996684350132626
Wall time: 4.63 s


In [27]:
% time get_model_stats(normalized_dev, dev_set_labels, logreg_clf)

T Positive: 396, F Positive: 270
F Negative: 5217, T Negative: 18010

The following metrics are on a scale of 0 to 1:
Model accuracy: 0.7703511488720546
Model precision: 0.5945945945945946
Model recall: 0.07055050774986638
Model F1 Score: 0.12613473483038698
Wall time: 1.2 s


##Random Forests Model

In the end, we found that a random forests model performed the best on this data. We did not achieve an accuracy that was wildly better than baseline, but given all of the metrics we used here, it seems that, on the whole, random forests performed well for this binary classification application. Running the classifier on features without normalization seemed to perform the best.

###Normalized Features Attempt

In [17]:
randfor_clf = RandomForestClassifier(n_estimators=40, max_depth=5)
% time randfor_clf.fit(normalized_train, train_set_labels)

Wall time: 7.48 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [20]:
% time get_model_stats(normalized_dev, dev_set_labels, randfor_clf)

T Positive: 1618, F Positive: 1546
F Negative: 3995, T Negative: 16734

The following metrics are on a scale of 0 to 1:
Model accuracy: 0.7680910726991169
Model precision: 0.511378002528445
Model recall: 0.28825939782647425
Model F1 Score: 0.3686908966617295
Wall time: 54.5 s


###Non-normalized Features Attempt

In [29]:
randfor_clf = RandomForestClassifier(n_estimators=40, max_depth=5)
% time randfor_clf.fit(featurized_train, train_set_labels)

Wall time: 4.35 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [30]:
% time get_model_stats(featurized_dev, dev_set_labels, randfor_clf)

T Positive: 842, F Positive: 344
F Negative: 4771, T Negative: 17936

The following metrics are on a scale of 0 to 1:
Model accuracy: 0.7859205625078475
Model precision: 0.7099494097807757
Model recall: 0.15000890789239266
Model F1 Score: 0.24768348286512723
Wall time: 55.9 s


In [135]:
# accuracy = correct predictions / total predictions
# It is the number of correct predictions made divided by the total number of predictions made

# precision = true positives / (true positives + false positives)
# Precision can be thought of as a measure of a classifiers exactness.
# A low precision can also indicate a large number of False Positives.

# recall = true positives / (true positives + false negatives)
# Recall can be thought of as a measure of a classifiers completeness.
# A low recall indicates many False Negatives.
# f1 = 2 * ((precision * recall) / (precision + recall))

#Running Model on Test Set 
(Non-normalized Features)

In [31]:
% time get_model_stats(featurized_test, test_labels, randfor_clf)

T Positive: 1143, F Positive: 574
F Negative: 4680, T Negative: 28373

The following metrics are on a scale of 0 to 1:
Model accuracy: 0.8488927236123095
Model precision: 0.6656959813628421
Model recall: 0.19629057187017002
Model F1 Score: 0.303183023872679
Wall time: 1min 20s


In [32]:
print(randfor_clf.feature_importances_)

[ 0.0033745   0.01645483  0.13675816  0.233239    0.12038536  0.07016558
  0.00295394  0.03919613  0.03617639  0.0198593   0.09358085  0.12706954
  0.03867808  0.01311949  0.00468971  0.02569395  0.0186052 ]


In [74]:
max_vals = sorted(randfor_clf.feature_importances_, reverse=True)[:6]

In [77]:
print("Top 5 features in order from most to least important:")
index = 1
for item in max_vals:
    item_index = np.where(randfor_clf.feature_importances_== item)[0][0]
    print("Rank: %-3d |  Feature: %-22s |   Importance score: %f" % (index, feature_dict[item_index], item))
    index += 1

Top 5 features in order from most to least important:
Rank: 1   |  Feature: total paragraphs       |   Importance score: 0.233239
Rank: 2   |  Feature: total characters       |   Importance score: 0.136758
Rank: 3   |  Feature: paragraphs per review  |   Importance score: 0.127070
Rank: 4   |  Feature: total cool votes       |   Importance score: 0.120385
Rank: 5   |  Feature: chars per review       |   Importance score: 0.093581
